In [1]:
import py5
import pandas as pd # pandasを使うためのインポート
import numpy as np # numpyを使うためのインポート
from pydub import AudioSegment
from pydub.generators import Sine 
from pydub.playback import play
import threading # スレッドを使うためのインポート
# スレッドとは、ある処理を別の流れで実行するための仕組み
# これにより、音を再生している間も他の処理が止まらないようにする

# グローバル変数（プログラム全体で使う変数）
# データ関連
df = None  # CSVデータを入れる変数
current_day = 0  # 今表示している日（0から始まる）
max_days = 0  # データの総日数

# 画面の中心座標
center_x = 400
center_y = 300

# 心臓のサイズ
base_radius = 50  # 基本の半径

# エフェクト用のリスト
particles = []  # パーティクル（飛び散る粒子）を入れるリスト
ripples = []    # 波紋を入れるリスト

# 音関連
bpm = 60  # 心拍数（1分間の拍数）
last_beat_time = 0  # 最後に心拍した時刻
beat_interval = 1000  # 心拍の間隔（ミリ秒）

# ========== 初期化関数 ==========

def load_data(csv_path):
    """CSVファイルを読み込んでデータを準備する"""
    global df, max_days
    
    # CSVファイルを読み込む
    df = pd.read_csv(csv_path)
    
    # 日付の列を日付型に変換
    df['日付'] = pd.to_datetime(df['日付'])
    
    # データの日数を記録
    max_days = len(df)
    
    # データを正規化（0〜1の範囲に調整）
    normalize_data()
    
    print(f"データを読み込みました: {max_days}日分")

def normalize_data():
    """データを0-1の範囲に正規化する（扱いやすくするため）"""
    global df
    
    # アクティブメンバー数を0-1に正規化
    min_active = df['アクティブなメンバー数（日次）'].min()
    max_active = df['アクティブなメンバー数（日次）'].max()
    df['active_normalized'] = (df['アクティブなメンバー数（日次）'] - min_active) / (max_active - min_active)
    
    # メッセージ数を0-1に正規化
    min_messages = df['投稿されたメッセージ数'].min()
    max_messages = df['投稿されたメッセージ数'].max()
    df['messages_normalized'] = (df['投稿されたメッセージ数'] - min_messages) / (max_messages - min_messages)

# ========== 更新関数（毎フレーム呼ばれる）==========

def update():
    """毎フレーム呼ばれる更新処理"""
    global current_day, bpm, beat_interval, last_beat_time
    
    current_time = py5.millis()  # 現在の時刻（ミリ秒）
    
    # 3秒ごとに次の日に進む（180フレーム = 3秒 * 60fps）
    if py5.frame_count % 180 == 0:
        current_day = (current_day + 1) % max_days  # 最後まで行ったら最初に戻る
        trigger_day_change()  # 日付が変わった時の処理
    
    # 現在の日のデータを取得
    current_data = df.iloc[current_day]
    
    # アクティビティレベルに応じてBPMを調整
    activity_level = current_data['active_normalized']
    bpm = 60 + int(activity_level * 120)  # 60〜180 BPMの範囲
    beat_interval = 60000 / bpm  # BPMから間隔を計算
    
    # 心拍のタイミングが来たら心拍する
    if current_time - last_beat_time > beat_interval:
        heartbeat(activity_level)
        last_beat_time = current_time
    
    # パーティクルと波紋を更新
    update_particles()
    update_ripples()

def heartbeat(intensity):
    """心拍エフェクトを作る"""
    global ripples
    
    # 波紋を追加
    ripples.append({
        'radius': base_radius,  # 開始時の半径
        'alpha': 255,          # 透明度（255が完全に不透明）
        'intensity': intensity  # 強さ
    })
    
    # 音を別スレッドで再生（メインの処理を止めないため）
    threading.Thread(target=lambda: play_heartbeat_sound(intensity)).start()

def play_heartbeat_sound(intensity):
    """心拍音を生成して再生する"""
    try:
        # 低い周波数の音でドクンという音を作る
        frequency = 60 + (intensity * 40)  # 60-100 Hz
        duration = 100 + int((1 - intensity) * 100)  # 100-200ms
        
        # サイン波で音を生成
        beat = Sine(frequency).to_audio_segment(duration=duration)
        beat = beat.fade_in(10).fade_out(50)  # フェードイン・アウト
        
        # 音量調整
        beat = beat - (20 - int(intensity * 15))
        
        # 再生
        play(beat)
    except:
        pass  # エラーが出ても続行

def trigger_day_change():
    """日付が変わった時の処理"""
    global particles
    
    # 現在の日のデータ
    current_data = df.iloc[current_day]
    
    # メッセージタイプの割合を取得
    dm_ratio = current_data['メッセージの割合、DM']
    public_ratio = current_data['メッセージの割合、パブリックチャンネル']
    
    # パーティクルの数を決定
    num_particles = int(current_data['messages_normalized'] * 50)
    
    # パーティクルを生成
    for i in range(num_particles):
        angle = py5.random(py5.TWO_PI)  # ランダムな角度
        speed = py5.random(1, 3)  # ランダムな速度
        
        # メッセージタイプによって色を決定
        rand = py5.random(1)
        if rand < dm_ratio:
            color = (100, 100, 255)  # 青：DM
        elif rand < dm_ratio + public_ratio:
            color = (100, 255, 100)  # 緑：パブリック
        else:
            color = (255, 100, 100)  # 赤：プライベート
        
        # パーティクルを追加
        particles.append({
            'x': center_x,
            'y': center_y,
            'vx': speed * py5.cos(angle),  # X方向の速度
            'vy': speed * py5.sin(angle),  # Y方向の速度
            'color': color,
            'life': 255  # 寿命（徐々に減る）
        })

def update_particles():
    """パーティクルを動かす"""
    global particles
    
    # パーティクルのコピーを作って繰り返し（削除しながら繰り返すため）
    for p in particles[:]:
        # 位置を更新
        p['x'] += p['vx']
        p['y'] += p['vy']
        
        # 寿命を減らす
        p['life'] -= 3
        
        # 寿命が尽きたら削除
        if p['life'] <= 0:
            particles.remove(p)

def update_ripples():
    """波紋を広げる"""
    global ripples
    
    for r in ripples[:]:
        # 半径を広げる
        r['radius'] += 2
        
        # 透明度を下げる
        r['alpha'] -= 5
        
        # 透明になったら削除
        if r['alpha'] <= 0:
            ripples.remove(r)

# ========== 描画関数 ==========

def draw_everything():
    """すべてを描画する"""
    # 現在の日のデータ
    current_data = df.iloc[current_day]
    
    # 背景を暗い色で塗りつぶす
    py5.background(20, 20, 30)
    
    # 情報テキストを表示
    draw_info_text(current_data)
    
    # 波紋を描画
    draw_ripples()
    
    # 中心の心臓を描画
    draw_heart(current_data)
    
    # パーティクルを描画
    draw_particles()
    
    # 曜日を表示
    draw_day_of_week(current_data)

def draw_info_text(data):
    """情報テキストを描画"""
    py5.fill(255)  # 白色
    py5.text_size(16)
    py5.text(f"日付: {data['日付'].strftime('%Y-%m-%d')}", 250, 30)
    py5.text(f"アクティブ: {data['アクティブなメンバー数（日次）']}人", 250, 50)
    py5.text(f"メッセージ: {data['投稿されたメッセージ数']}件", 250, 70)
    py5.text(f"BPM: {bpm}", 250, 90)

def draw_ripples():
    """波紋を描画"""
    py5.no_fill()  # 塗りつぶさない
    
    for r in ripples:
        # 透明度を計算
        alpha = r['alpha'] * r['intensity']
        py5.stroke(255, 255, 255, alpha)  # 白い線
        py5.stroke_weight(2)  # 線の太さ
        py5.circle(center_x, center_y, r['radius'] * 2)

def draw_heart(data):
    """中心の心臓を描画"""
    # アクティビティレベルでサイズを決定
    activity = data['active_normalized']
    radius = base_radius + activity * 30
    
    # 脈動アニメーション（心拍に合わせて大きさが変わる）
    beat_phase = (py5.millis() - last_beat_time) / beat_interval
    pulse = 1 + 0.1 * py5.sin(beat_phase * py5.PI)
    
    # 心臓を描画
    py5.fill(255, 100, 100, 200)  # 赤っぽい色
    py5.no_stroke()
    py5.circle(center_x, center_y, radius * 2 * pulse)

def draw_particles():
    """パーティクルを描画"""
    for p in particles:
        py5.fill(*p['color'], p['life'])  # 色と透明度を設定
        py5.circle(p['x'], p['y'], 5)  # 小さな円を描画

def draw_day_of_week(data):
    """曜日を表示"""
    day_of_week = data['日付'].strftime('%A')  # 曜日を英語で取得
    py5.fill(255)
    py5.text_size(24)
    py5.text_align(py5.CENTER)
    py5.text(day_of_week, center_x, 550)

# ========== メイン関数 ==========
def setup():
    """最初に一度だけ実行される初期化処理"""
    py5.size(800, 600)  # ウィンドウサイズ
    py5.color_mode(py5.RGB)  # カラーモード
    font = py5.create_font("Hiragino Sans", 16)  # Mac用日本語フォント
    py5.text_font(font)
    
    # データを読み込む
    load_data('slack_time.csv')

def draw():
    """毎フレーム実行される描画処理"""
    if df is not None:  # データが読み込まれていたら
        update()  # 更新処理
        draw_everything()  # 描画処理

def key_pressed():
    """キーが押された時の処理"""
    global current_day
    
    if py5.key == ' ':
        # スペースキーで一時停止/再開
        if py5.is_looping():
            py5.no_loop()
        else:
            py5.loop()
    elif py5.key == 'r':
        # Rキーでリセット
        current_day = 0

# プログラムを実行
py5.run_sketch()

Importing py5 on macOS but the necessary Jupyter macOS event loop has not been activated. I'll activate it for you, but next time, execute `%gui osx` before importing this library.


データを読み込みました: 397日分
